In [1]:
using Distributions
using Rocket
using ReactiveMP
using BenchmarkTools

import Base: show

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1278
┌ Warning: Package ReactiveMP does not have Rocket in its dependencies:
│ - If you have ReactiveMP checked out for development and have
│   added Rocket as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ReactiveMP
└ Loading Rocket into ReactiveMP from project dependency, future warnings for ReactiveMP are suppressed.


In [5]:
function filtering_model_with_logging()
    model = Model(; message_gate = DefaultMessageGate() + LoggerMessageGate())
    
    x_prev = datavar(model, :x_prior, NormalMeanVariance{Float64})
    add_1  = constvar(model, :add_1, 1.0)
    x      = randomvar(model, :x)
    
    x_prev_add = make_node(model, +, x_prev, add_1, x)
    
    noise = constvar(model, :noise, NormalMeanVariance(0.0, 200.0))
    y     = datavar(model, :y, PointMass{Float64})
    
    add_x_and_noise = make_node(model, +, x, noise, y)
    
    activate!(model)
    
    return x_prev, x, y
end

function filtering(data)
    N = length(data)
    
    x_prev, x, y = filtering_model_with_logging()
    
    marginals = Vector{NormalMeanVariance{Float64}}()
    
    subscription = subscribe!(getmarginal(x), (t) -> begin
        update!(x_prev, getdata(t))   
        push!(marginals, getdata(t))
    end)
    
    update!(x_prev, NormalMeanVariance(0.0, 10000.0))
    
    for d in data
        update!(y, d)
    end
    
    unsubscribe!(subscription)
    
    return marginals
end

filtering (generic function with 1 method)

In [6]:
N = 10
data = collect(1:N) + sqrt(200.0) * randn(N);

In [7]:
filtering(data);

From variable in2 of node + => NormalMeanVariance{Float64}(μ=-1.0, v=10000.0)
From variable out of node + => NormalMeanVariance{Float64}(μ=-14.473175444608133, v=200.0)
From variable in2 of node + => NormalMeanVariance{Float64}(μ=-15.208995533929542, v=196.07843137254903)
From variable out of node + => NormalMeanVariance{Float64}(μ=-8.878616463604644, v=200.0)
From variable in2 of node + => NormalMeanVariance{Float64}(μ=-13.075144509016225, v=99.009900990099)
From variable out of node + => NormalMeanVariance{Float64}(μ=9.611081326964005, v=200.0)
From variable in2 of node + => NormalMeanVariance{Float64}(μ=-6.563149199088997, v=66.22516556291389)
From variable out of node + => NormalMeanVariance{Float64}(μ=-23.213407365415883, v=200.0)
From variable in2 of node + => NormalMeanVariance{Float64}(μ=-11.70500446434444, v=49.751243781094516)
From variable out of node + => NormalMeanVariance{Float64}(μ=-22.972160512568603, v=200.0)
From variable in2 of node + => NormalMeanVariance{Float64}(μ